A biblioteca padrão de C++ disponibiliza diversas funções para utilização de *RNG*s (cabeçalho <random> - documentação neste link). Se você quisesse sortear números aleatórios inteiros entre -2 e 5 quais funções usaria?

In [ ]:
%%writefile programa.cpp 
#include <iostream>
#include <random>
#include <chrono>
using namespace std;

int main() {
  unsigned seed = chrono::system_clock::now().time_since_epoch().count();
  default_random_engine generator(seed);
  uniform_int_distribution<int> distribution(-2,5);
  cout << distribution(generator); // gera número
}

Overwriting programa.cpp


In [ ]:
!g++ -o programa programa.cpp

In [ ]:
!./programa

3

 E se você quisesse sortear um número real entre 0 e 1?

In [ ]:
%%writefile programa.cpp 
#include <iostream>
#include <random>
#include <chrono>
using namespace std;

int main() {
  unsigned seed = chrono::system_clock::now().time_since_epoch().count();
  default_random_engine generator(seed);
  uniform_real_distribution<double> distribution(0.0, 1.0);
  cout << distribution(generator); // gera número
}

Overwriting programa.cpp


In [ ]:
!g++ -o programa programa.cpp

In [ ]:
!./programa

0.392501

Agora que você já consegue gerar números aleatórios, vamos implementar nossa primeira versão de uma heurística aleatorizada.

Mochila mais leve:

In [ ]:
%%writefile programa.cpp 

#include<iostream>
#include<vector>
#include<algorithm>
using namespace std;

struct item {
    int id;
    double peso;
    double valor;
};

bool my_compare(item a, item b){
    return a.peso < b.peso; // ordenando pelo mais leve
}

int main() {
    int n = 0;
    int W = 0;
    vector<item> mochila;
    vector<item> items;
    cin >> n >> W;
    items.reserve(n);
    double peso, valor;
    for(int i = 0; i < n; i++){
        cin >> peso;
        cin >> valor;
        items.push_back({i, peso, valor});
    }
    //ordenar
    sort(items.begin(), items.end(), my_compare);
    peso = 0;
    valor = 0;
    for(auto& el : items){
        if(el.peso + peso <= W){
            mochila.push_back(el);
            peso += el.peso;
            valor += el.valor;
        }
    }
    //ordenando para imprimir
    sort(mochila.begin(), mochila.end(), [](auto& i, auto&j){return i.id < j.id;});
    cout << peso << " " << valor << " 0" << endl;
    for(auto& el: mochila){
        cout << el.id << " ";
    }

    return 0;
}

Overwriting programa.cpp


In [ ]:
!g++ -o programa programa.cpp

In [ ]:
!./programa < "in1.txt" > "out1.txt"

Adicionaremos a seguinte variação na nossa heurística: a cada passo de seleção temos 25% de chance de selecionar um objeto aleatório que ainda não foi utilizado. Ou seja, cada passo do algoritmo segue a seguinte regra

Faça um sorteio aleatório
Com probabilidade 75% pegue o próximo objeto não selecionado de acordo com a heurística (mais leve ou mais caro)
Com probabilidade 25% selecione um objeto qualquer dos que não foram analisados ainda.
Note que não mudamos o próximo elemento ao fazer a seleção aleatória. Adote seed=10 nesta tarefa.

Dica: agora é possível que eu olhe um produto mais de uma vez. Você precisará checar isso no seu programa!

In [ ]:
%%writefile programa.cpp 

#include <iostream>
#include <vector>
#include <cmath>
#include<fstream>
#include <bits/stdc++.h>

using namespace std;


struct item{
    int id;
    double peso;
    double valor;
};
bool my_compare(item a, item b); //assinatura

int main() {
    default_random_engine generator;
    generator.seed(10);

    int n = 0;
    int w =0;
    uniform_real_distribution<double> distribution(0.0,1.0);
    vector<item> mochila;
    vector<item> items;

    cin >> n >> w; 
    items.reserve(n);

    
    items.reserve(n);
    double peso, valor;

    
    for(int i = 0 ; i<n; i++){
        cin >> peso;
        cin >> valor;
        items.push_back({i,peso,valor});
     }
     
     //sinvariante - elementos ordenado pelo peso (eh um ponto de certeza)
     sort(items.begin(),items.end(), [](auto& i, auto& j){return i.valor > j.valor;}); 
     
     peso = 0;
     valor = 0;
     int i = 1;
     for(auto& el: items){
         
            if(el.peso + peso <= w){
                mochila.push_back(el);
                peso += el.peso;
                valor +=el.valor;
            
            }
            if (distribution(generator) > 0.75 && i<n){
                uniform_int_distribution<int> distribution(i,n-1);
                int p = distribution(generator);
                if (items[p].peso + peso <= w){
                    mochila.push_back(items[p]);
                    peso = peso + items[p].peso;
                    valor = valor + items[p].valor;

                    items.erase(items.begin()+p-1);
                    n=n-1;

                }
            }
            i=i+1;
     }

     //ordenando para imprimir
     sort(mochila.begin(),mochila.end(),[](auto&i, auto&j){return i.id < j.id;});
     cout << peso << " " << valor << " 0" << "\n";

     for (auto& el : mochila){
         cout << el.id << " ";
     }

     cout << peso << " " << valor << " 0" << endl;
     for(auto& el: mochila){
         cout << el.id << " ";
     }
    
    
    return 0;
}
bool my_compare(item a,item b){
    return a.peso < b.peso; //ordem crescente se fosse crescente seria a < b 
}

Overwriting programa.cpp


In [ ]:
!g++ -o programa programa.cpp

In [ ]:
!./programa < "in1.txt"

8 241 0
1 5 7 8 241 0
1 5 7 